In [1]:
import numpy as np

In [15]:
def hms2dec(h, m, s):
    """Converts right ascension from HMS to decimal degrees
    
    Arguments:
        h {integer} -- Hours
        m {integer} -- Minutes
        s {integer} -- Seconds
    
    Returns:
        integer -- Converted rigth ascension
    """
    result = (15*(h+m/60+s/(60*60)))
    return result

def dms2dec(d, arcm, arcs):
    """Convert decimal degrees to dms
    
    Arguments:
        d {integer} -- Degrees
        arcm {integer} -- Arcminutes
        arcs {float} -- Arcseconds
    
    Returns:
        integer -- Degrees to dms
    """
    
    if d < 0:
        result = -1*((-1*d) +(arcm/60)+(arcs/(60*60)))
    else:
        result = d+(arcm/60)+(arcs/(60*60))
    return result

def import_bss():
    res = []
    data = np.loadtxt('bss.dat', usecols=range(1, 7))
    for i, row in enumerate(data, 1):
        hms = hms2dec(row[0], row[1], row[2])
        dms = dms2dec(row[3], row[4], row[5])
        res.append((i, hms, dms))
    return res

def import_super():
    res = []
    data = np.loadtxt('super.csv', delimiter=',', skiprows=1, usecols=[0, 1])
    for i, row in enumerate(data, 1):
        res.append((i, row[0], row[1]))
    return res

def angular_dist(r1, d1, r2, d2):
    """Calculate angular distance
    
    Arguments:
        r1 {float} -- Radius 1
        d1 {float} -- Distance 1
        r2 {float} -- Radius 1
        d2 {float} -- Distance 2
    
    Returns:
        integer -- Calculate distance between two objects
    """
    # Convert radians
    r1 = np.radians(r1)
    d1 = np.radians(d1)
    r2 = np.radians(r2)
    d2 = np.radians(d2)
    
    a = np.sin(np.abs(d1 - d2)/2)**2
    b = np.cos(d1)*np.cos(d2)*np.sin(np.abs(r1 - r2)/2)**2
    d = 2*np.arcsin(np.sqrt(a + b))
    
    #Return to degrees
    return np.degrees(d)

def crossmatch(cat1, cat2, max_radius):
    """Calculate crossmatches two catalogues within a maximum distance.
    
    Arguments:
        cat1 {dataframe} -- 1 Catalogue
        cat2 {dataframe} -- 2 Catalogue
        max_radius {float} -- Max radius distance
    
    Returns:
        list -- list of matches and non-matches
    """
    matches = []
    no_matches = []
    for id1, ra1, dec1 in cat1:
        closest_dist = np.inf
        closest_id2 = None
        for id2, ra2, dec2 in cat2:
            dist = angular_dist(ra1, dec1, ra2, dec2)
            if dist < closest_dist:
                closest_id2 = id2
                closest_dist = dist
        if closest_dist > max_radius:
            no_matches.append(id1)
        else:
            matches.append((id1, closest_id2, closest_dist))
    return matches, no_matches

In [16]:
# You can use this to test your function.
# Any code inside this `if` statement will be ignored by the automarker.
if __name__ == '__main__':
    bss_cat = import_bss()
    super_cat = import_super()

    # First example in the question
    max_dist = 40/3600
    matches, no_matches = crossmatch(bss_cat, super_cat, max_dist)
    print(matches[:3])
    print(no_matches[:3])
    print(len(no_matches))

    # Second example in the question
    max_dist = 5/3600
    matches, no_matches = crossmatch(bss_cat, super_cat, max_dist)
    print(matches[:3])
    print(no_matches[:3])
    print(len(no_matches))

[(1, 2, 0.00010988610938710059), (2, 4, 0.0007649845967242495), (3, 5, 0.00020863352870707666)]
[5, 6, 11]
9
[(1, 2, 0.00010988610938710059), (2, 4, 0.0007649845967242495), (3, 5, 0.00020863352870707666)]
[5, 6, 11]
40
